### Read data

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.lines as mlines
from defdap.quat import Quat
import defdap.ebsd as ebsd
import defdap.hrdic as hrdic
from skimage import transform as tf
from skimage import io, measure
from IPython.display import display
import scipy as scipy
import peakutils
from ipywidgets import FloatProgress
from IPython.display import display
%matplotlib osx
f = FloatProgress(min=0, max=100)
display(f)

#Load in DIC maps
DicFilePath = "newdata/"
DicMap = hrdic.Map(DicFilePath, "irr_003.TXT")
DicMap.setPatternPath("irr_000_8bin.jpg", 2)
#DicMap.setCrop(xMin=100,xMax=100,yMin=100,yMax=200)
DicMap.setCrop(xMin=150,xMax=293,yMin=115,yMax=214)
f.value += 25

DicMap2 = hrdic.Map(DicFilePath, "nonirr_003.TXT")
DicMap2.setPatternPath("nonirr_000_8bin.jpg", 2)
#DicMap2.setCrop(xMin=115,xMax=150,yMin=80,yMax=100)
DicMap2.setCrop(xMin=131,xMax=166,yMin=100,yMax=99)
f.value += 25

pixelinm=78.125e-9

#Load in EBSD map and calculate misorientation
EbsdMap = ebsd.Map("newdata/irr_02_cleaned", "hexagonal")
EbsdMap.loadSlipSystems("defdap/slip_systems/hexagonal_withca.txt", cOverA=1.593)
EbsdMap.buildQuatArray()
EbsdMap.findBoundaries(boundDef = 4)
EbsdMap.boundaries = scipy.misc.imread('newdata/newebsd/i_boundaries.bmp').astype(int)
EbsdMap.boundaries[EbsdMap.boundaries == 0] = 0
EbsdMap.boundaries[EbsdMap.boundaries == 255] = -1
EbsdMap.findGrains(minGrainSize= 50)
EbsdMap.calcGrainMisOri(calcAxis = False)
f.value += 25

EbsdMap2 = ebsd.Map("newdata/newebsd/ni-clean6", "hexagonal")
EbsdMap2.loadSlipSystems("defdap/slip_systems/hexagonal_withca.txt", cOverA=1.593)
EbsdMap2.buildQuatArray()
EbsdMap2.findBoundaries(boundDef = 4)
EbsdMap2.boundaries = scipy.misc.imread('newdata/newebsd/ni_boundaries.bmp').astype(int)
EbsdMap2.boundaries[EbsdMap2.boundaries == 0] = 0
EbsdMap2.boundaries[EbsdMap2.boundaries == 255] = -1
EbsdMap2.findGrains(minGrainSize= 100)
EbsdMap2.calcGrainMisOri(calcAxis = False)
f.value += 25

Widget Javascript not detected.  It may not be installed or enabled properly.


IndexError: index 721 is out of bounds for axis 1 with size 721

### Export boundaries

In [ ]:
#EXPORT BOUNDARY TO FILE
from skimage import morphology as mph
import matplotlib as mpl
cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['white', 'black'], 256)
cmap1._init()
cmap1._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)
boundariesImage = -EbsdMap2.boundaries
#boundariesImage = mph.binary_dilation(boundariesImage)
plt.imshow(boundariesImage, cmap=cmap1, vmin=0, vmax=1)

import scipy.misc
scipy.misc.imsave('newdata/newebsd/ni_boundaries.bmp', boundariesImage)

### Define homologous points

In [ ]:
DicMap2.homogPoints = np.array((  (342, 397), (134, 560), (1182, 975), (1124, 1032), 
                                (158, 1062), (268, 1112), (80, 786), (84, 220), (726, 956), 
                                (596, 1148), (1042, 504), (1220, 326), (1182, 620), (1060, 872), 
                                (904, 1121), (652, 629), (501, 910), (502, 648), (716, 296) , 
                                (430, 239), (890, 690), (879, 79), (242, 64), (568, 48) ))
EbsdMap2.homogPoints = np.array((  (167, 232), (80, 286), (484, 469), (462, 489), (79, 472), 
                                 (121, 494), (54, 368), (67, 160), (305, 446), (253, 516), 
                                 (440, 289), (513, 229), (491, 337), (438, 428), (373, 519), 
                                 (281, 325), (218, 423), (226, 327), (317, 204), (203, 174), 
                                 (378, 353), (385, 128), (135, 106), (262, 106) ))
DicMap.homogPoints = np.array(( (379, 204), (55, 1072), (559, 318), (800, 1074), (1136, 747), (1114, 1097), (1141, 562), (1048, 174), (796, 222), (435, 103), (153, 485), (153, 228), (325, 1110), (454, 674), (906, 570), (1050, 866), (1038, 374), (459, 481), (788, 657), (334, 860), (119, 681), (765, 98), (96, 41), (1184, 14), (693, 806), (680, 1134), (1172, 114), (734, 463), (666, 330) ))
EbsdMap.homogPoints = np.array(( (196, 94), (46, 404), (263, 136), (327, 419), (464, 300), (445, 434), (471, 231), (451, 93), (353, 107), (219, 62), (102, 186), (109, 99), (155, 424), (208, 259), (380, 230), (430, 342), (439, 161), (216, 192), (336, 259), (159, 327), (83, 257), (342, 66), (94, 42), (506, 48), (293, 312), (280, 443), (499, 78), (321, 189), (301, 139) ))  

In [ ]:
#DicMap.homogPoints = np.array((   ))
#EbsdMap.homogPoints = np.array((   ))

#DicMap2.setHomogPoint(display="pattern")
#EbsdMap2.setHomogPoint()

DicMap.setHomogPoint(display="pattern")
EbsdMap.setHomogPoint()

In [ ]:
print(DicMap.homogPoints)
print(EbsdMap.homogPoints)

### Link maps and find grains

In [ ]:
DicMap.linkEbsdMap(EbsdMap)
DicMap.findGrains(minGrainSize=100)

In [ ]:
DicMap2.linkEbsdMap(EbsdMap2)
DicMap2.setEbsdShift(xShift=0, yShift=0)
DicMap2.findGrains(minGrainSize=100)

### Plot max shear

In [9]:
DicMap.plotMaxShear(plotGBs=True,vmin=0,vmax=10,plotColourBar=True,dilateBoundaries=True)

In [51]:
EbsdMap.plotMisOriMap(plotGBs=True,vmax=8)

In [ ]:
plt.imshow(EbsdMap.boundaries)
#EbsdMap.plotEulerMap()

In [ ]:
#fig, ((ax0, ax1)) = plt.subplots(1, 2, figsize=(20, 10))
ax0 = plt.subplot(121)
ax1 = plt.subplot(122, sharex=ax0, sharey=ax0)
ax0.imshow(DicMap.crop(DicMap.max_shear),vmax=0.1)
ax1.imshow(DicMap.crop(0.5*DicMap.f12-0.5*DicMap.f21),cmap='coolwarm',vmin=-0.08,vmax=0.08)

In [ ]:
shear_map_filt=DicMap.crop(DicMap.f21)-DicMap.crop(DicMap.f12)
shear_map_filt2=DicMap.crop(DicMap.max_shear)
plt.figure()
gs = gridspec.GridSpec(1, 3)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1], sharex=ax0, sharey=ax0)
ax2=plt.subplot(gs[2], sharex=ax0, sharey=ax0)
ax0.imshow(shear_map_filt,cmap='coolwarm',vmin=-0.12,vmax=0.12)
ax1.imshow(shear_map_filt2,cmap='viridis',vmin=0.00,vmax=0.08)
bseImage = plt.imread(DicMap.patternImPath)
bseImage=scipy.misc.imresize(bseImage,0.5)
ax2.imshow(DicMap.crop(bseImage, binned=0.5), cmap='gray', vmin=0, vmax=255)

In [ ]:
from matplotlib.colors import PowerNorm

rot_map=DicMap.crop(DicMap.max_shear)
plt.figure()
gs = gridspec.GridSpec(1, 1)
ax0=plt.subplot(gs[0])
img=ax0.imshow(rot_map,cmap='viridis',vmin=0,vmax=0.2, norm=PowerNorm(gamma=0.8))
plt.colorbar(img, ax=ax0, label="Effective shear strain (%)")

In [ ]:
EbsdMap.calcAverageGrainSchmidFactors
EbsdMap.plotAverageGrainSchmidFactorsMap()

### Histograms

In [ ]:
print("Max Exx irr: {0:.2f}%".format(np.max(DicMap.crop(DicMap.f11))*100))
print("Max Exx ni: {0:.2f}%".format(np.max(DicMap2.crop(DicMap2.f11))*100))

print("Max shear irr: {0:.2f}%".format(np.max(DicMap.crop(DicMap.max_shear))*100))
print("Max shear ni: {0:.2f}%".format(np.max(DicMap2.crop(DicMap2.max_shear))*100))

print("Min shear irr: {0:.2f}%".format(np.min(DicMap.crop(DicMap.max_shear))*100))
print("Min shear ni: {0:.2f}%".format(np.min(DicMap2.crop(DicMap2.max_shear))*100))

print("Mean Exx irr: {0:.2f}%".format(np.mean(DicMap.crop(DicMap.f11))*100))
print("Mean Exx ni: {0:.2f}%".format(np.mean(DicMap2.crop(DicMap2.f11))*100))

print("Mean shear irr: {0:.2f}%".format(np.mean(DicMap.crop(DicMap.max_shear))*100))
print("Mean shear ni: {0:.2f}%".format(np.mean(DicMap2.crop(DicMap2.max_shear))*100))

print(np.percentile(DicMap.crop(DicMap.max_shear)*100,99.9))
print(np.percentile(DicMap2.crop(DicMap2.max_shear)*100,99.9))

print("Points irr: {0}".format(len(DicMap.crop(DicMap.max_shear)[0])*len(DicMap.crop(DicMap.max_shear)[1])))
print("Points ni: {0}".format(len(DicMap2.crop(DicMap2.max_shear)[0])*len(DicMap2.crop(DicMap2.max_shear)[1])))

In [ ]:
irr_crop= DicMap.crop(DicMap.max_shear)
irr_crop= DicMap.crop(DicMap.max_shear) / np.mean(DicMap.crop(DicMap.max_shear))
ni_crop= DicMap2.crop(DicMap2.max_shear)
ni_crop= DicMap2.crop(DicMap2.max_shear) / np.mean(DicMap2.crop(DicMap2.max_shear))

hist, bins = np.histogram(irr_crop, bins=800,range=[0,60],density=True)
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
#plt.show()
np.savetxt('irr.csv', (hist), delimiter=',')

hist, bins = np.histogram(ni_crop, bins=800,range=[0,60],density=True)
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
#plt.show()
np.savetxt('nonirr.csv', (hist), delimiter='\n')

In [ ]:
gs = gridspec.GridSpec(1, 3)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1],sharex=ax0,sharey=ax0)
ax2=plt.subplot(gs[2],sharex=ax0,sharey=ax0)
ax0.imshow(DicMap.crop(DicMap.max_shear),vmax=0.1)
ax1.imshow(DicMap.crop(DicMap.x_map),vmax=0.01)
ax2.imshow(DicMap.crop(DicMap.y_map),vmax=0.01)

In [ ]:
gb_binary=(DicMap.boundaries<0)
plt.imshow((gb_binary.astype(int)))
grain_labels=measure.label(gb_binary)
print(grain_labels)

In [8]:
fig,ax= plt.subplots()
ax.imshow(DicMap.boundaries)

for grainID in range(0,130):
    
    xmiddle=(DicMap.grainList[grainID].extremeCoords[2]+DicMap.grainList[grainID].extremeCoords[0])/2
    ymiddle=(DicMap.grainList[grainID].extremeCoords[3]+DicMap.grainList[grainID].extremeCoords[1])/2
    ax.text(xmiddle, ymiddle, grainID,fontsize=10)

IndexError: list index out of range

### RDR Calculation

In [ ]:
import matplotlib as mpl
from skimage import morphology as mph
bigulist=[]
bigvlist=[]
biggradlist=[]
bigrsquaredlist=[]

#Define the lines (x0,y0)->(x1,y1)
#x0=np.array([338,321,314,350,357,308,327])
#y0=np.array([87,96,110,82,77,120,152])
#x1=np.array([423,414,397,427,428,385,361])
#y1=np.array([162,177,184,150,138,187,180])
#x0-=5
#y0-=5
#x1+=5
#y1+=5

#x0=np.array([18,97,82])
#y0=np.array([902,883,967])
#x1=np.array([220,245,235])
#y1=np.array([706,734,819])

x0=np.array([951,1006,1036])
y0=np.array([100,118,168])
x1=np.array([939,991,1027])
y1=np.array([127,143,187])

minx=np.amin([x0,x1])
minx-=50
maxx=np.amax([x0,x1])
maxx+=50
miny=np.amin([y0,y1])
miny-=50
maxy=np.amax([y0,y1])
maxy+=50

gs = gridspec.GridSpec(1, 3)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1])

ax0.imshow(DicMap.crop(DicMap.max_shear),vmax=0.1,cmap='viridis')
#ax0.plot(x,y,'-')
ax0.set_title('Effective shear strain')
ax0.set_xlim(minx,maxx)
ax0.set_ylim(maxy,miny)
scalebar = ScaleBar(pixelinm) # 1 pixel = 0.2 meter
ax0.add_artist(scalebar)
cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['white', 'white'], 256)
cmap1._init()
cmap1._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)
boundariesImage = -DicMap.boundaries
boundariesImage = mph.binary_dilation(boundariesImage)
ax0.imshow(boundariesImage, cmap=cmap1, vmin=0, vmax=1)

for x0,y0,x1,y1 in zip(x0,y0,x1,y1):
    grad=(y1-y0)/(x1-x0)
    pergrad=-1/grad
    profile_length=np.sqrt((y1-y0)**2+(x1-x0)**2)
    num = np.round(1+profile_length/np.sqrt(grad**2+1))
    x, y = np.linspace(x0, x1, num), np.linspace(y0, y1, num)
    x=np.round(x)
    y=np.round(y)

    for i,j in zip(x,y):
        x0new=i+np.sqrt(8/(pergrad**2+1))*np.sign(grad)
        y0new=j-np.sqrt(8/(1/pergrad**2+1))
        x1new=i-np.sqrt(8/(pergrad**2+1))*np.sign(grad)
        y1new=j+np.sqrt(8/(1/pergrad**2+1))
        profile_length=np.sqrt((y1new-y0new)**2+(x1new-x0new)**2)
        num = np.round(profile_length)
        xnew, ynew = np.linspace(x0new, x1new, num), np.linspace(y0new, y1new, num)
        xnew=np.round(xnew)
        ynew=np.round(ynew)
        ax0.plot(xnew,ynew, '-',lw=2)
        ulist=[]
        vlist=[]
        for xnew,ynew in zip(xnew,ynew):
            xnew.astype(int)
            ynew.astype(int)
            ulist.append(DicMap.crop(DicMap.x_map)[np.int_(xnew),np.int_(ynew)])
            vlist.append(DicMap.crop(DicMap.y_map)[np.int_(xnew),np.int_(ynew)])
        ulistav=np.mean(ulist)
        vlistav=np.mean(vlist)
        ulist-=ulistav
        vlist-=vlistav
        #plt.plot(ylist,xlist)
        slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(vlist,ulist)
        if r_value**2>0.8:
            bigulist.extend(ulist)
            bigvlist.extend(vlist)
            biggradlist.append(slope)
            bigrsquaredlist.append(r_value**2)
    ax0.plot([x0, x1], [y0, y1], 'rx-',lw=2)
    
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(bigvlist,bigulist)

ax1.scatter(bigvlist,bigulist,marker='x')
minx=np.min(bigvlist)
maxx=np.max(bigvlist)
miny=slope*minx+intercept
maxy=slope*maxx+intercept
ax1.plot([minx, maxx],[miny,maxy], '-')
ax1.set_xlabel('v-centered')
ax1.set_ylabel('u-centered')
print("Slope = {0:.3f} +- {1:.3f}".format(slope,std_err))
print("R-squared = {0:.3f}".format(r_value**2))
ax2=plt.subplot(gs[2])
ax2.plot(np.arange(len(biggradlist)),biggradlist)
#ax2.scatter(biggradlist,bigrsquaredlist)

print(np.average(biggradlist))
print(np.std(biggradlist)/np.sqrt(len(biggradlist)))


### Locate a grain of interest

In [ ]:
DicMap.locateGrainID(displaySelected=True, vmax=10)
maxStrain = np.max(DicMap.max_shear)
print("Max effective shear strain: {0:.3f} %\n".format(maxStrain))

In [ ]:
DicMap2.grainList[DicMap2.currGrainId].plotMaxShear(vmax=10)

In [ ]:
DicMap2.grainList[DicMap2.currGrainId].ebsdGrain.plotMisOri(vmax=5)

In [ ]:
plt.hist([142.17,141.25,139.54,66.6,133.47,46.65,137.14,69.6,49.3,41.11,47.38,121.48,57.29,138.75,133.04,69.4,34.63,138.03,140.71,93,68.52,35.61, 59.46,65.63,128.387,58.7,57.46,67.82,121.68,133.83,47.09,121.52,124.29,137,132.72,47.19,136.29,56.71,37.04,44.09,135.32,52.96,55.96,129.47,30.22,45.67,48.6,141.42,50.45,143.11,125.84,130.39,56.72,53.5,129.07,143.87,58.41,38.02,72.93,119.45,55.33,49.53,49.82,131.17,141.66,143.04],bins=18,range=[0,180])

### Grain info

In [ ]:
from builtins import print
Grain=DicMap.grainList[DicMap.currGrainId]
ebsdGrain = Grain.ebsdGrain

print("Area: {} px^2".format(len(ebsdGrain)*0.25))
print("width: {0}".format(Grain.extremeCoords[2]-Grain.extremeCoords[0]))
print("height: {0}".format(Grain.extremeCoords[3]-Grain.extremeCoords[1]))
minStrain = np.min(Grain.maxShearList)*100
meanStrain = np.mean(Grain.maxShearList)*100
maxStrain = np.max(Grain.maxShearList)*100
print("Min effective shear strain: {0:.3f} %".format(minStrain))
print("Mean effective shear strain: {0:.3f} %".format(meanStrain))
print("Max effective shear strain: {0:.3f} %\n".format(maxStrain))

ebsdGrain.calcAverageSchmidFactors(loadVector=np.array([1, 0, 0]))
ebsdGrain.calcSlipTraces()
print("Euler angles: ")
#print("",ebsdGrain.refOri.eulerAngles()*180/np.pi, "\n")

angle=ebsdGrain.refOri.eulerAngles()[1]*180/np.pi

if ebsdGrain.refOri.eulerAngles()[1]*180/np.pi > 90:
    angle=180-angle

print("Angle from c-axis?: {0:.1f} degrees\n".format(angle))

schdFacs = ebsdGrain.averageSchmidFactors
slipTraces = ebsdGrain.slipTraces
for ssGroup, sfGroup, slipTrace in zip(EbsdMap.slipSystems, schdFacs, slipTraces):
    stAngle = np.arctan(slipTrace[1]/slipTrace[0]) *180/np.pi
    if stAngle < 0: stAngle += 180
    if stAngle > 180: stAngle -= 180
    print("Slip Plane Normal {0:s}\tSlip trace: ({1:.3f}, {2:.3f})\tAngle: {3:.1f}".
          format(ssGroup[0].slipPlaneLabel, slipTrace[0], slipTrace[1], stAngle))
    for ss, sf in zip(ssGroup, sfGroup):
        slipDirSample=ebsdGrain.refOri.conjugate.transformVector(ss.slipDir)
        print("\tSlip Direction {0:s}\tSchmid factor: {1:.3f}\tRDR: {2:.3f}".
              format(ss.slipDirLabel, sf,slipDirSample[0]/slipDirSample[1]))
        

### Grain Stats

In [ ]:
from builtins import print
print("GrainID\tArea\t<ESS>\tEuler1\tEuler2\tEuler3\tAngC\tMaxESS\tMinESS")
print("\t(px^2)\t(%)\t(deg)\t(deg)\t(deg)\t(deg)\t(%)\t(%)")
meanStrains=[]
angC=[]
maxStrains=[]
minStrains=[]
errors=[0,0]
maxschmidlist=[]

SelDicMap=DicMap

for x in range(0,len(SelDicMap.grainList)):
     
    ebsdGrain = SelDicMap[x].ebsdGrain
    meanStrain = np.mean(SelDicMap.grainList[x].maxShearList)*100
    maxStrain = np.max(SelDicMap.grainList[x].maxShearList)*100
    minStrain = np.min(SelDicMap.grainList[x].maxShearList)*100
    ebsdGrain.calcAverageSchmidFactors(loadVector=np.array([1, 0, 0]))
    ebsdGrain.calcSlipTraces()
    Euler1=ebsdGrain.refOri.eulerAngles()[0]*180/np.pi
    Euler2=ebsdGrain.refOri.eulerAngles()[1]*180/np.pi
    Euler3=ebsdGrain.refOri.eulerAngles()[2]*180/np.pi
    angle=ebsdGrain.refOri.eulerAngles()[1]*180/np.pi
    meanStrains.append(meanStrain)
    maxStrains.append(maxStrain)
    minStrains.append(minStrain)
    if angle > 90:
        angle=180-angle
    angC.append(angle)
    prismschmid=np.max((ebsdGrain.averageSchmidFactors[1],ebsdGrain.averageSchmidFactors[2],ebsdGrain.averageSchmidFactors[3]))
    #prismschmid=ebsdGrain.averageSchmidFactors[0]
    maxschmidlist.append(prismschmid)
    
    print("{0}\t{1:.2f}\t{2:.2f}\t{3:.2f}\t{4:.2f}\t{5:.2f}\t{6:.2f}\t{7:.2f}\t{8:.2f}\t{9:.3f}\t".format(x,len(ebsdGrain)*0.25,meanStrain,Euler1,Euler2,Euler3,angle,maxStrain,minStrain,prismschmid))
  
lowe=[i - j for i, j in zip(meanStrains, minStrains)]
highe=[k - l for k, l in zip(maxStrains, meanStrains)]
errors[0]=lowe
errors[1]=highe
#fig=plt.errorbar(angC,meanStrains,yerr=errors, marker='x',linestyle="None",capsize=5)
#plt.scatter(angC,[k / l for k, l in zip(maxStrains, meanStrains)])
plt.show()
plt.hist(maxschmidlist)
#prism ni 0.457 i 0.455
#basal ni 0.161 i 0.174
print(np.mean(maxschmidlist))

### Grain profile

In [ ]:
def plot_profile(def_map,start_point,end_point):
    x0,y0  = start_point[0], start_point[1] # These are in _pixel_ coordinates!!
    x1,y1 = end_point[0], end_point[1]
    profile_length=np.sqrt((y1-y0)**2+(x1-x0)**2)
    num = np.round(profile_length)
    x, y = np.linspace(x0, x1, num), np.linspace(y0, y1, num)

    # Extract the values along the line, using cubic interpolation\n",
    zi = scipy.ndimage.map_coordinates(np.transpose(def_map), np.vstack((x,y)))
    return zi

In [ ]:
Grain=DicMap2.grainList[DicMap2.currGrainId]
unzippedCoordlist = list(zip(*Grain.coordList))
x0 = min(unzippedCoordlist[0])
y0 = min(unzippedCoordlist[1])
xmax = max(unzippedCoordlist[0])
ymax = max(unzippedCoordlist[1])
grainMaxShear = np.full((ymax - y0 + 1, xmax - x0 + 1), 0, dtype=float)


for coord, maxShear in zip(Grain.coordList, Grain.maxShearList):
    grainMaxShear[coord[1] - y0, coord[0] - x0] = maxShear
    
plt.imshow(grainMaxShear)

point1=[35,35]
point2=[230,150]

plt.scatter(35,35)
plt.scatter(230,150)

corr_profile2=plot_profile(grainMaxShear,point1,point2)

#fig2=plt.scatter(range(len(corr_profile2)),corr_profile2,linestyle='-')

#f=open('/Users/mbcx9rt5/Dropbox (Research Group)/python/corr_profile.txt', 'wb')
np.savetxt('corr_profile.txt',corr_profile2,fmt='%.6f')
#f.close()

In [ ]:
Grain=DicMap.grainList[DicMap.currGrainId]
unzippedCoordlist = list(zip(*Grain.coordList))
x0 = min(unzippedCoordlist[0])
y0 = min(unzippedCoordlist[1])
xmax = max(unzippedCoordlist[0])
ymax = max(unzippedCoordlist[1])
grainMaxShear = np.full((ymax - y0 + 1, xmax - x0 + 1), 0, dtype=float)

for coord, maxShear in zip(Grain.coordList, Grain.maxShearList):
    grainMaxShear[coord[1] - y0, coord[0] - x0] = maxShear

print(grainMaxShear)
    
corr_profile2=plot_profile(grainMaxShear,[90,90],[260,260])

#f=open('/Users/mbcx9rt5/Dropbox (Research Group)/python/corr_profile.txt', 'wb')
np.savetxt('corr_profile2.txt',corr_profile2,fmt='%.6f')
#f.close()

In [ ]:
newvalues = [x for x in DicMap2.grainList[DicMap2.currGrainId].maxShearList if x != 0]

hist, bins = np.histogram(newvalues, bins=800,range=[0,0.6],density=True)
#hist=np.log10(hist*100)
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
np.savetxt('grainnirr.csv', (hist), delimiter=',')

### Grain info

In [ ]:
#Calculate FWHM given peak positions (indexes) and array (x,y) 
def FWHM(indexes, Xorig,Yori):
    fwhmList=[]   #Define empty array for resulting FWHMs
    indexes+=20
    Xorig=np.arange(220)
    Yorig=np.concatenate((Yori[160:180],Yori,Yori[0:20]))
    
    for indexes in indexes:
        X=Xorig[indexes-18:indexes+18]
        Y=Yorig[indexes-18:indexes+18]
        half_max = max(Y) / 2.
        #find when function crosses line half_max (when sign of diff flips)
        #take the 'derivative' of signum(half_max - Y[])
        d = np.sign(half_max - np.array(Y[0:-1])) - np.sign(half_max - np.array(Y[1:]))
        #plot(X,d) #if you are interested
        #find the left and right most indexes
        left_idx = X[np.where( d > 0.1 )]
        right_idx = X[np.where( d < -0.1 )]
        FWHM=0
        if len(right_idx)==1 and len(left_idx)==1:
            FWHM=right_idx - left_idx #return the difference (full width)
        fwhmList.append(np.int(FWHM))
    fwhmList=np.asarray(fwhmList)
    return fwhmList

In [ ]:
SelDicMap=DicMap2

for grainID in range (99,100):
    ebsdGrain = SelDicMap[grainID].ebsdGrain
    minStrain = np.min(SelDicMap.grainList[grainID].maxShearList)*100
    meanStrain = np.mean(SelDicMap.grainList[grainID].maxShearList)*100
    maxStrain = np.max(SelDicMap.grainList[grainID].maxShearList)*100
    medianStrain = np.median(SelDicMap.grainList[grainID].maxShearList)*100
    ebsdGrain.calcAverageSchmidFactors(loadVector=np.array([1, 0, 0]))
    ebsdGrain.calcSlipTraces()
    np.set_printoptions(precision=3)
##### GENERATE MAX SHEARS
    x0, y0, xmax, ymax = SelDicMap[grainID].extremeCoords
    grainMaxShear = np.full((ymax - y0 + 1, xmax - x0 + 1), np.nan, dtype=float)
    for coord, maxShear in zip(SelDicMap.grainList[grainID].coordList, SelDicMap.grainList[grainID].maxShearList):
        grainMaxShear[coord[1] - y0, coord[0] - x0] = maxShear
##### WRITE TEXT  
        B = []
    def print(str):
        global B
        B.append(str)
    #from builtins import print
    print("Grain ID: {0:.0f}".format(grainID))
    print("Area: {} um^2".format(len(SelDicMap.grainList[grainID].coordList)*0.006103515625))
    print("Min effective shear strain: {0:.3f} %".format(minStrain))
    print("Mean effective shear strain: {0:.3f} %".format(meanStrain))
    print("Max effective shear strain: {0:.3f} %".format(maxStrain))
    print("Median effective shear strain: {0:.3f} %\n".format(medianStrain))
    print(ebsdGrain.refOri.eulerAngles()*180/np.pi)
    angle=ebsdGrain.refOri.eulerAngles()[1]*180/np.pi
    if ebsdGrain.refOri.eulerAngles()[1]*180/np.pi > 90:
        angle=180-angle
    print("Angle from c-axis?: {0:.1f} degrees\n".format(angle))
##### SHOW MAX SHEAR
    fig, ((ax0, ax1, ax2, ax3), (ax4, ax5, ax6, ax7)) = plt.subplots(2, 4, figsize=(20, 10))
    im=ax1.imshow(grainMaxShear,cmap='viridis',vmin=0,vmax=0.1)
    fig.colorbar(im, ax=ax1,label="Effective shear strain (%)")
    scalebar = ScaleBar(pixelinm) # 1 pixel = 0.2 meter
    ax1.add_artist(scalebar)
    ax1.axis('off')
##### CALCULATE RADON
    grainMaxShear[np.isnan(grainMaxShear)]=0
    sin_map = tf.radon(grainMaxShear,circle=False,theta=list(range(90,180)) + list(range(90)))
    profile = np.max(sin_map, axis=0)
    x = np.arange(180)
    y = profile
    base = peakutils.baseline(y, 2)
    y=y-base
    indexes = peakutils.indexes(y, thres=0.2, min_dist=20)
    peaks = peakutils.interpolate(x, y, ind=indexes)
    shearBandAngles = peaks
    breadths=FWHM(indexes,x,y)
    for ID in range(0,len(shearBandAngles)):
        if shearBandAngles[ID] > 180:
            shearBandAngles[ID]-=180
    print("Angles detected: {}".format(shearBandAngles))
    print("FWHMs: {}".format(breadths))
###### CALCULATE SLIP TRACE ANGLES
    schdFacs = ebsdGrain.averageSchmidFactors
    slipTraces = ebsdGrain.slipTraces
    for ssGroup, sfGroup, slipTrace in zip(EbsdMap.slipSystems, schdFacs, slipTraces):
        stAngle = np.arctan(slipTrace[1]/slipTrace[0]) *180/np.pi
        stAngle += 2
        if stAngle < 0: stAngle += 180
        if stAngle > 180: stAngle -= 180
        print("{0:s}     Angle: {1:.1f}".
              format(ssGroup[0].slipPlaneLabel, stAngle))
        for ss, sf in zip(ssGroup, sfGroup):
            print("{0:s}     {1:.3f}".format(ss.slipDirLabel, sf))

    colours = ["red", "blue", "blue", "blue", "green", "green", "green", "green", "green", "green"]
    xPos = int((xmax - x0) / 2)
    yPos = int((ymax - y0) / 2)
##### GENERATE FIGURES
    ax2.axis('off')
    ax3.axis('off')
    ax0.imshow(sin_map)
    ax4.plot(x,y)
    for xc in peaks:
        ax4.axvline(x=xc)
##### PLOTTING RADON AND FWHM
    plotRadon='False'
    if plotRadon=='True':
        shearBandAngles = -(180-shearBandAngles) * np.pi / 180
        breadths = -breadths * np.pi / 180
        shearBandVectors = np.array((np.cos(shearBandAngles), np.sin(shearBandAngles)))
        ax3.quiver(xPos, yPos, shearBandVectors[0], shearBandVectors[1], scale=0.9, pivot="middle",
                           color='orange', headwidth=1, headlength=0)
        fwhmAngles = []
        for breadth, angle in zip(breadths, shearBandAngles):
            fwhmAngles.append(angle+(breadth/2))
            fwhmAngles.append(angle-(breadth/2))
        fwhmVectors = np.array((np.cos(fwhmAngles), np.sin(fwhmAngles)))
        ax3.quiver(xPos, yPos, fwhmVectors[0], fwhmVectors[1], scale=0.9, pivot="middle",
                           color='orange', headwidth=1, headlength=0)
###### PLOT SLIP TRACE ANGLES
    for i, slipTrace in enumerate(slipTraces):
        colour = colours[len(colours) - 1] if i >= len(colours) else colours[i]
        ax3.quiver(xPos, yPos, slipTrace[0], slipTrace[1], scale=1, pivot="middle",
                              color=colour, headwidth=1, headlength=0)
###### MAKE HISTOGRAM
    newvalues = [x for x in SelDicMap.grainList[grainID].maxShearList if x != 0]
    hist, bins = np.histogram(newvalues, bins=800,range=[0,0.6],density=True)
    #hist=np.log10(hist*100)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
    ax7.plot(center, hist, marker='x',color='b',linestyle='none')
    #align='center', width=width
    ax7.set_xscale("log", nonposx='clip')
    ax7.set_yscale("log", nonposy='clip')
    #f=open('/Users/mbcx9rt5/Dropbox (Research Group)/python/grain_freq_profile.txt', 'wb')
    #combined = np.vstack((x,y)).T
    #np.savetxt(f,combined,delimiter=" ", fmt="%s")
    #f.close()
###### SHOW TEXT   
    plt.text(0.5, 0.5,'\n'.join([ str(myelement) for myelement in B ]), horizontalalignment='center',verticalalignment='center',transform=ax2.transAxes,fontsize=6)
    plt.tight_layout(pad=0)
    from builtins import print
    #print ('\n'.join([ str(myelement) for myelement in B ]))
    #print ('\n')
    #print("\n")

### Detect slip system

In [46]:
ID=[0,1,14,11,9,18,19,22,23,24,26,27,27,30,32,36,38,37,40,40,39,42,44,44,46,48,48,52,52,55,57,60,60,61,61,63,64,66,70,69,71,71,72,72,73,74,75,77,78,79,82,82,83,84,86,85,87,88,89,91,93,92,95,96,100,101,103,104,105,106,107,107,108,111,113]
DetAngles=[66.6,139.54,141.25,142.17,49.3,69.6,137.14,46.65,133.47,138.75,57.29,121.48,61.02,47.38,41.11,93,140.71,138.03,34.63,149.94,69.4,133.04,65.63,108.79,59.46,35.61,140.55,68.52,120.35,133.83,67.82,57.46,42.59,58.7,118.09,128.387,136.29,47.19,132.72,137,124.29,68.47,121.52,57.26,47.09,129.47,55.96,52.96,135.32,44.09,37.04,151.96,56.71,130.39,125.84,143.11,50.45,141.42,48.6,45.67,58.41,143.87,129.07,53.5,56.72,141.66,131.17,49.82,49.53,55.33,119.45,59.17,72.93,38.02,143.04]
SelDicMap=DicMap

#ID=[2,8,10,11,12,13,21,22,24,25,27,30,36,36,39,41,42,45,47,48,49,52,52,53,54,55,56,57,58,59,60,61,62,66,70,72,75,75,76,78,81,84,93,95,98,98,99,99,103,104,105,109,110,112]
#DetAngles=[58.86,150.9,40.1,51.55,139.17,132.75,160,53.27,80.04,134.59,148.31,142.35,35.42,152.84,57.28,126.01,44.15,120.89,123.87,121.69,74.34,32.94,151.83,53.17,45.67,51.5,131.95,64.81,119.54,159.72,146,130.51,128.56,145.64,130.18,38.35,116.87,67.55,50.78,53.13,127,131.95,66.05,134.08,59.16,112.88,130.27,69.69,143.31,111.33,124.76,36.87,53.78,50.1]
#SelDicMap=DicMap2

EbsdMap.calcAverageGrainSchmidFactors()
EbsdMap2.calcAverageGrainSchmidFactors()

bigdev=[]
bigschmid=[]
bigsystem=[]
bigmean=[]
bigmax=[]
counter=[0,0,0,0]
for i in range (0,len(ID)):
    grainID=ID[i]
    ebsdGrain = SelDicMap[grainID].ebsdGrain
    ebsdGrain.calcSlipTraces()
    schdFacs = ebsdGrain.averageSchmidFactors
    slipTraces = ebsdGrain.slipTraces
    
    #print("GrainID: {0}".format(grainID))
    #print(DetAngles[grainID])

    angles=[]
    systems=[]
    schmids=[]
    detected=[]
    dev=[]
    
    for ssGroup, sfGroup, slipTrace in zip(EbsdMap.slipSystems, schdFacs, slipTraces):
        stAngle = np.arctan(slipTrace[1]/slipTrace[0]) *180/np.pi
        if stAngle < 0: stAngle += 180
        if stAngle > 180: stAngle -= 180
        stAngle += 2.6
        #print("{0:s}     Angle: {1:.1f}".format(ssGroup[0].slipPlaneLabel, stAngle))
        angles.append(stAngle)
        systems.append(ssGroup[0].slipPlaneLabel)
        for ss, sf in zip(ssGroup, sfGroup):
            schmids.append(sf)
    systems=['Ba','Pr1','Pr2','Pr3','Py1','Py2','Py3','Py4','Py5','Py6']
    
    for x in range(0,10):
        if angles[x]-5 <= DetAngles[i] <= angles[x]+5:
            detected.append(systems[x])
            dev.append(DetAngles[i]-angles[x])
    if len(dev)==0:
        counter[0]+=1
    if len(dev)==1:
        bigdev.append(dev)
        counter[1]+=1
        if 'Pr' in detected[0]:
            counter[2]+=1
            bigsystem.append('Pr')
            bigschmid.append(np.max((ebsdGrain.averageSchmidFactors[1],ebsdGrain.averageSchmidFactors[2],ebsdGrain.averageSchmidFactors[3])))
            bigmean.append(np.mean(SelDicMap.grainList[i].maxShearList)*100)
            bigmax.append(np.max(SelDicMap.grainList[i].maxShearList)*100)
        if 'Py' in detected[0]:
            counter[3]+=1
            bigsystem.append('Py')
            bigschmid.append(np.max((ebsdGrain.averageSchmidFactors[4],ebsdGrain.averageSchmidFactors[5],ebsdGrain.averageSchmidFactors[6],ebsdGrain.averageSchmidFactors[7],ebsdGrain.averageSchmidFactors[8])))
            bigmean.append(np.mean(SelDicMap.grainList[i].maxShearList)*100)
            bigmax.append(np.max(SelDicMap.grainList[i].maxShearList)*100)
    #print(detected)
    #print(dev)

print("Prismatic: {0} ({2:.1f}%) / Pyramidal: {1} ({3:.1f}%)".format(counter[2],counter[3],counter[2]*100/(counter[2]+counter[3]),counter[3]*100/(counter[2]+counter[3])))

print("One solution: {0} / No solution: {1} / Multiple: {2} / Total: {3}".format(counter[1],counter[0],len(ID)-counter[1]-counter[0],len(ID)))
print(np.mean(bigdev))

hist, bins = np.histogram(bigdev, bins=40,range=[-10,10],density=False);
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)

hist, bins = np.histogram(bigschmid, bins=40,range=[0,0.5],density=False);
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
#plt.bar(center, hist, align='center', width=width)

#plt.scatter(bigschmid,bigmax)
#plt.xlim(0, 0.5)

Prismatic: 27 (81.8%) / Pyramidal: 6 (18.2%)
One solution: 33 / No solution: 6 / Multiple: 36 / Total: 75
0.360423091959


### Plot profile across max shear map

In [ ]:
def plot_profile(def_map,start_point,end_point):
    x0,y0  = start_point[0], start_point[1] # These are in _pixel_ coordinates!!
    x1,y1 = end_point[0], end_point[1]
    profile_length=np.sqrt((y1-y0)**2+(x1-x0)**2)
    num = np.round(profile_length)
    x, y = np.linspace(x0, x1, num), np.linspace(y0, y1, num)

    # Extract the values along the line, using cubic interpolation
    zi = scipy.ndimage.map_coordinates(np.transpose(def_map), np.vstack((x,y)))
    plt.figure(figsize=(20,6))
    gs = gridspec.GridSpec(1, 2, height_ratios=[1]) 
    ax0=plt.subplot(gs[0])
    ax1=plt.subplot(gs[1])
    ax0.imshow(def_map,vmin=-0.00,vmax=0.2,interpolation='bilinear',cmap='viridis');
    ax0.plot([x0, x1], [y0, y1], 'rx-',lw=2);
    ax0.axis('image');
    ax1.plot(zi)
    ax1.axis('tight')
    return zi;

In [ ]:
#corr_profile=plot_profile(DicMap.max_shear,[583,627],[445,772])
#corr_profile2=plot_profile(DicMap2.max_shear,[820,607],[911,718])
corr_profile2=plot_profile(DicMap2.crop(DicMap2.max_shear),[160,426],[361,525])
#f=open('/Users/mbcx9rt5/Dropbox (Research Group)/python/corr_profile.txt', 'wb')
#np.savetxt(f,corr_profile,fmt='%.6f')
#f.close()
f=open('/Users/mbcx9rt5/Dropbox (Research Group)/python/corr_profile.txt', 'wb')
np.savetxt(f,corr_profile2,fmt='%.6f')
f.close()

### 2-point correlation GRAIN

In [ ]:
unzippedCoordlist = list(zip(DicMap.grainList[DicMap.currGrainId].coordList))
x0 = min(unzippedCoordlist[0])
y0 = min(unzippedCoordlist[1])
xmax = max(unzippedCoordlist[0])
ymax = max(unzippedCoordlist[1])
grainMaxShear = np.full((ymax - y0 + 1, xmax - x0 + 1), np.nan, dtype=float)

for coord, maxShear in zip(DicMap.grainList[DicMap.currGrainId].coordList, DicMap.grainList[DicMap.currGrainId].maxShearList):
            grainMaxShear[coord[1] - y0, coord[0] - x0] = maxShear

x=acorr_map(grainMaxShear)
plt.imshow(x,cmap='viridis')

### 2-point correlation MAP

In [ ]:
def acorr_map(def_map,c_range=[]):
    acorr=(np.fft.fft2(def_map)*np.conjugate(np.fft.fft2(def_map)))
    ashift=np.fft.fftshift(acorr)
    corr_map=np.log(np.fft.fftshift((np.abs(np.fft.ifft2(ashift)))))
    if c_range==[]:
        plt.imshow(corr_map, interpolation='nearest', cmap='viridis');
    else:
        plt.imshow(corr_map, interpolation='nearest', cmap='viridis',
                   vmin=c_range[0], vmax=c_range[1]);
    return corr_map

x=acorr_map(DicMap.max_shear[200:1200,200:1200])
y=acorr_map(DicMap2.max_shear[200:1200, 200:1200])

plt.imshow(DicMap.max_shear[200:1200,200:1200])

plt.figure(figsize=(10,5))
gs = gridspec.GridSpec(1, 2)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1])
ax0.imshow(y,cmap='viridis')
ax1.imshow(x,cmap='viridis')

plt.figure(figsize=(10,5))
gs = gridspec.GridSpec(1, 2)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1])
ax0.imshow(y[450:550,450:550],cmap='viridis')
ax1.imshow(x[450:550,450:550],cmap='viridis')

### Plot max shear for isolated grain with slip traces

In [ ]:
DicMap.grainList[DicMap.currGrainId].calcSlipTraces()
DicMap.grainList[DicMap.currGrainId].plotMaxShear(plotSlipTraces=True)

#DicMap.grainList[DicMap.currGrainId].calcSlipTraces(correctAvOri=False)
#DicMap.grainList[DicMap.currGrainId].plotMaxShear(plotSlipTraces=True)

In [ ]:
scale=0.02
fig = plt.figure()
plt.subplot(221)

fig = plt.imshow(DicMap.f11,cmap='coolwarm',vmin=-scale,vmax=scale);
cbar = plt.colorbar(pad=0.02)
cbar.set_label(label="f11")

plt.subplot(222)

fig = plt.imshow(DicMap.f12,cmap='coolwarm',vmin=-scale,vmax=scale);
cbar = plt.colorbar(pad=0.02)
cbar.set_label(label="f12")

plt.subplot(223)

fig = plt.imshow(DicMap.f21,cmap='coolwarm',vmin=-scale,vmax=scale);
cbar = plt.colorbar(pad=0.02)
cbar.set_label(label="f21")

plt.subplot(224)

fig = plt.imshow(DicMap.f22,cmap='coolwarm',vmin=-scale,vmax=scale);
cbar = plt.colorbar(pad=0.02)
cbar.set_label(label="f22")

plt.show()

### Display maps with homologous points

In [ ]:
EbsdMap.plotBoundaryMap()
plt.scatter(x=EbsdMap.homogPoints[:, 0], y=EbsdMap.homogPoints[:, 1], c='y', s=60)

In [ ]:
DicMap.plotMaxShear()
plt.scatter(x=DicMap.homogPoints[:, 0], y=DicMap.homogPoints[:, 1], c='y', s=60)

In [ ]:
rawImagePath = "irr1/raw.png"

rawImage = io.imread(rawImagePath)

plt.figure()
plt.imshow(rawImage)
plt.scatter(x=DicMap.homogPoints[:, 0], y=DicMap.homogPoints[:, 1], c='y', s=60)
plt.show()

shear_map_filt=DicMap2.f11[200:1200,200:1200]
shear_map_filt2=DicMap2.max_shear[200:1200,200:1200]
plt.figure()
gs = gridspec.GridSpec(1, 2)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1])
ax0.imshow(shear_map_filt,cmap='viridis',vmin=-0.01,vmax=0.1)
ax1.imshow(shear_map_filt2,cmap='viridis',vmax=0.1)

### SPP MSS/Rot

In [ ]:
shear_map_filt=DicMap2.f21[540:610,590:670]-DicMap2.f12[540:610,590:670]
shear_map_filt2=DicMap2.max_shear[540:610,590:670]
plt.figure()
gs = gridspec.GridSpec(1, 2)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1])
ax0.imshow(shear_map_filt,cmap='coolwarm',vmin=-0.08,vmax=0.1)
ax1.imshow(shear_map_filt2,cmap='viridis',vmin=0.00,vmax=0.08)
#ax0.title('max shear', fontsize=20)

#f11_profile=plot_profile(shear_map_filt2,[30,50],[44,66])
#f11_profile=plot_profile(shear_map_filt,[30,50],[44,66])
maxStrain = np.max(DicMap.max_shear)*100
print("Max effective shear strain: {0:.3f} %\n".format(maxStrain)) 

In [ ]:
shear_map_filt=DicMap.f21[840:1210,890:1270]-DicMap.f12[840:1210,890:1270]
shear_map_filt2=DicMap.max_shear[840:1210,890:1270]
plt.figure()
gs = gridspec.GridSpec(1, 2)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1])
ax0.imshow(shear_map_filt,cmap='coolwarm',vmin=-0.0008,vmax=0.001)
ax1.imshow(shear_map_filt2,cmap='viridis',vmin=0.0000,vmax=0.0008)
#ax0.title('max shear', fontsize=20)

#f11_profile=plot_profile(shear_map_filt2,[30,50],[44,66])
#f11_profile=plot_profile(shear_map_filt,[30,50],[44,66])
maxStrain = np.max(DicMap.max_shear)*100
print("Max effective shear strain: {0:.3f} %\n".format(maxStrain)) 

In [ ]:
### Smaller

In [ ]:
cropmap=DicMap2.max_shear[200:1200,200:1200]
plt.figure()
gs = gridspec.GridSpec(1, 1)
ax0=plt.subplot(gs[0])
ax0.imshow(cropmap,cmap='coolwarm',vmin=-0.08,vmax=0.1)

maxStrain = np.max(cropmap)*100
print("Max effective shear strain: {0:.3f} %\n".format(maxStrain)) 

In [ ]:
shear_map_filt=DicMap.f21[1000:1200,850:1200]-DicMap2.f12[1000:1200,850:1200]
shear_map_filt2=DicMap.f11[1000:1200,850:1200]
plt.figure()
gs = gridspec.GridSpec(1, 2)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1])
ax0.imshow(shear_map_filt,cmap='viridis',vmin=0.00,vmax=0.08)
ax1.imshow(shear_map_filt2,cmap='viridis',vmin=0.00,vmax=0.08)

### Threshold

In [ ]:
#99th irr 19.05   nonirr 6.16
#98th irr 9.61 nonirr 5.37

threshold=0.02
shear_map_filt=DicMap2.crop(DicMap2.max_shear)>threshold

import scipy.misc
scipy.misc.imsave('newdata/nirr_threshold.bmp', DicMap2.crop(DicMap2.max_shear)>threshold)

sin_map = tf.radon(shear_map_filt, circle=False)
profile_filt=np.max(sin_map,axis=0)
plt.figure()
gs = gridspec.GridSpec(1, 2)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1])
strain_title='Threshold: {:2.3f}'.format(threshold)
ax0.imshow(shear_map_filt,cmap='binary')
ax0.set_title(strain_title)
ax1.plot(profile_filt)

In [ ]:
threshold=0.117
shear_map_filt=DicMap.crop(DicMap.max_shear)>threshold

import scipy.misc
scipy.misc.imsave('newdata/irr_threshold.bmp', DicMap.crop(DicMap.max_shear)>threshold)

sin_map = tf.radon(shear_map_filt, circle=False)
profile_filt=np.max(sin_map,axis=0)
plt.figure(figsize=(30,15))
gs = gridspec.GridSpec(1, 2)
ax0=plt.subplot(gs[0])
ax1=plt.subplot(gs[1])
strain_title='Threshold: {:2.3f}'.format(threshold)
ax0.imshow(shear_map_filt,cmap='viridis')
ax0.set_title(strain_title)
ax1.plot(profile_filt)

###### DicMap.buildNeighbourNetwork()

In [ ]:
DicMap.displayNeighbours()